# Соединение таблиц

## 1. Знакомимся с данными

Таблица teams с данными о командах
Название столбца
	
Содержимое столбца
id 	id команды
api_id 	ключ на таблицу matches
long_name 	полное название команды
short_name 	сокращённое название команды


Таблица matches с данными о матчах
Название столбца
	
Содержимое столбца
id 	id матча
season 	сезон
date 	дата матча
home_team_api_id 	api_id домашней команды, ключ на таблицу teams
away_team_api_id 	api_id гостевой команды, ключ на таблицу teams
home_team_goals 	количество голов домашней команды
away_team_goals 	количество голов гостевой команды

Напишите запрос, который выведет сезон (season), общее количество забитых мячей домашними (total_home_goals) и гостевыми (total_away_goals) командами.
Отсортируйте по столбцу с сезоном в порядке возрастания.

select 
season,
sum(home_team_goals) as total_home_goals,
sum(away_team_goals) as total_away_goals
from sql.matches
group by season
order by season asc



## 2. Соединение таблиц по ключу

### Объединяем таблицы без операторов

Существует несколько способов соединения таблиц. Мы познакомимся со всеми основными операторами, которые используются для этих нужд, но начнём с простого метода объединения таблиц — без операторов.

Чтобы соединить две таблицы между собой, достаточно записать названия таблиц через запятую в разделе from. 
SELECT *
FROM
    sql.teams,
    sql.matches
__
кол-во строк:
select
count(*)
FROM
    sql.teams,
    sql.matches
__

В данном случае соединение таблиц не даёт практической пользы: мы получили очень много записей, которые никак не можем интерпретировать, потому что команды не соответствуют матчам.

Ключ — это поле (столбец) в таблице, которое позволяет однозначно идентифицировать запись (строку).

Чтобы соединить таблицы и получить данные о домашней команде по каждому матчу, добавим условие
where home_team_api_id = api_id.

SELECT * /*выбор всех полей в таблице*/
FROM
    sql.teams, /*таблица с командами*/
    sql.matches /*таблица с матчами*/
WHERE home_team_api_id = api_id /*условие: home_team_api_id таблицы matches равен api_id таблицы teams*/

Аналогично можем получить данные о гостевых командах: необходимо изменить условие на
where away_team_api_id = api_id.


SELECT *
FROM
    sql.teams,
    sql.matches
WHERE away_team_api_id = api_id

Итак, мы только что объединили таблицы по ключу.

Как правило, названия ключей имеют «хвост», который позволяет их идентифицировать: например, _id, _rk, _cd, _pk (от primary_key), _fk (от foreign_key) и другие.

Обратите внимание! В данном датасете ключ api_id таблицы teams может быть использован в разных значениях. Его можно использовать для того, чтобы получить информацию о домашней (home) или гостевой (away) команде.


С помощью известного нам запроса получим названия команд, игравших домашние матчи, и счёт матчей.
SELECT 
    long_name, 
    home_team_goals,
    away_team_goals
FROM
    sql.teams,
    sql.matches
WHERE home_team_api_id = api_id

SELECT 
    long_name, 
    home_team_goals,
    away_team_goals
FROM
    sql.teams,
    sql.matches
WHERE home_team_api_id = api_id

## 3. Знакомимся с JOIN

JOIN — это оператор SQL, который позволяет соединять таблицы по условию.
В качестве примера используем запрос из предыдущего юнита.
SELECT 
    long_name,
    home_team_goals,
    away_team_goals
FROM
    sql.teams,
    sql.matches
WHERE home_team_api_id = api_id

и запишем его с использованием JOIN.
SELECT 
    long_name,
    home_team_goals,
    away_team_goals
FROM    
    sql.teams
JOIN sql.matches on home_team_api_id = api_id


### Синтаксис


Оператор JOIN упрощает процесс соединения таблиц.

Его синтаксис можно представить следующим образом:

SELECT
        столбец1,
	столбец2,
	...
FROM
	таблица1
JOIN таблица2 ON условие

Порядок присоединения таблиц в данном случае не важен — результат будет одинаковым.

С помощью JOIN можно соединить и более двух таблиц.
В таблицах, которые мы соединяем, могут быть одинаковые названия столбцов.

→ К примеру, столбец id есть и в таблице matches, и в таблице teams. Такой запрос не будет обработан.
Можно указать, откуда мы хотим запросить данные, записав название таблицы перед столбцом через точку.


SELECT
    teams.id
FROM 
    sql.teams
JOIN sql.matches ON home_team_api_id = api_id

Можно также выбрать и столбец из таблицы matches.

SELECT
    matches.id
FROM 
    sql.teams
JOIN sql.matches on home_team_api_id = api_id

Зачастую названия таблиц слишком длинные, так что использовать их неудобно.

Упростить обращение к различным таблицам можно, присвоив им сокращённые названия — алиасы (от англ. alias).

Вместо алиасов с кавычками рекомендуется давать простое короткое название на латинице, без специальных символов и пробелов. Если таблиц немного и все названия начинаются с разных букв, можно присваивать алиасы по первой букве.

__ 
Напишите запрос, который выведет два столбца: id матча (match_id) и id домашней команды (team_id), — а затем отсортируйте по id матча в порядке возрастания значений.

select
matches.id as match_id,
teams.id as team_id
from sql.teams
join sql.matches on home_team_api_id = api_id
order by match_id asc
___

Давайте с помощью запроса SQL получим таблицу, содержащую:

        название домашней команды;
        количество забитых домашней командой голов;
        количество забитых гостевой командой голов;
        название гостевой команды.

SELECT
    h.long_name "домашняя команда", /*столбец long_name таблицы h*/
    m.home_team_goals "голы домашней команды", /*столбец home_team_goals таблицы m*/
    m.away_team_goals "голы гостевой команды", /*столбец away_team_goals таблицы m*/
    a.long_name "гостевая команда" /*столбец long_name таблицы a*/
FROM
    sql.matches m /*таблица matches с алиасом m*/
    JOIN sql.teams h ON m.home_team_api_id = h.api_id /*оператор соединения таблиц; таблица teams с алиасом h; условие: home_team_api_id таблицы m равен api_id таблицы h*/
    JOIN sql.teams a ON m.away_team_api_id = a.api_id /*оператор соединения таблиц; таблица teams с алиасом a; условие: away_team_api_id таблицы m равен api_id таблицы a*/

__
Напишите запрос, который выведет столбцы: id матча, короткое название домашней команды (home_short), короткое название гостевой команды (away_short).
Отсортируйте запрос по возрастанию id матча.

SELECT
    m.id as match_id,
    t.short_name as home_short,
    g.short_name as away_short
FROM
    sql.matches m
    JOIN sql.teams t ON m.home_team_api_id = t.api_id
    JOIN sql.teams g ON m.away_team_api_id = g.api_id
    
order by match_id asc


## 4. Фильтрация и агрегатные функции

### Работа с объединёнными таблицами

Фильтрация данных

К соединённым таблицам применимы функции фильтрации данных.

Например, можно вывести id матчей, в которых команда Arsenal была гостевой.

SELECT 
    m.id
FROM
    sql.teams t
    JOIN sql.matches m ON m.away_team_api_id = t.api_id
WHERE long_name = 'Arsenal'

### Принципиальное отличие фильтрации данных по соединённым таблицам от аналогичного действия по одиночным таблицам заключается в том, что, фильтруя записи одной таблицы, мы также будем фильтровать и записи другой таблицы, поскольку соединённые на уровне запроса таблицы по сути являются единой таблицей.
Например, результат запроса
SELECT 
	m.id id_1,
	m.season,
	t.id id_2,
	t.long_name
FROM
	sql.teams t
JOIN sql.matches m ON m.away_team_api_id = t.api_id

___
Напишите запрос, который выведет полное название команды (long_name), количество голов домашней команды (home_goal) и количество голов гостевой команды (away_goal) в матчах, где домашней командой были команды с коротким названием ‘GEN’.
Отсортируйте запрос по id матча в порядке возрастания.

SELECT
	t.long_name long_name,
	m.home_team_goals home_goal,
	m.away_team_goals away_goal
FROM	
	sql.matches m
JOIN sql.teams t ON t.api_id = m.home_team_api_id
WHERE
	t.short_name = 'GEN'
ORDER BY m.id

Также мы можем отфильтровать записи сразу по двум таблицам.
Например, можно оставить только записи, в которых короткое название домашней команды GEN и матчи сезона 2008/2009.

SELECT *
FROM    
    sql.matches m
    JOIN sql.teams t on t.api_id = m.home_team_api_id
WHERE
    t.short_name = 'GEN'
    AND m.season = '2008/2009'

Напишите запрос, чтобы вывести id матчей, короткое название домашней команды (home_short), короткое название гостевой команды (away_short) для матчей сезона 2011/2012, в которых участвовала команда с названием Liverpool.
Отсортируйте по id матча в порядке возрастания.

SELECT
    m.id,
	t1.short_name home_short,
    t2.short_name away_short
FROM	
	sql.matches m
JOIN sql.teams t1 ON t1.api_id = m.home_team_api_id
join sql.teams t2 on t2.api_id = m.away_team_api_id
WHERE
    m.season = '2011/2012'
    and (t1.long_name = 'Liverpool' or t2.long_name = 'Liverpool')
order by m.id ASC

### Агрегация данных

К соединённым таблицам также применимы любые агрегатные функции — самые важные функции для анализа данных.

Например, мы можем вывести сумму голов по командам для матчей, где команда выступала в гостях.
SELECT
    t.long_name,
    SUM(m.home_team_goals) + SUM(m.away_team_goals) match_goals
FROM
    sql.matches m
    JOIN sql.teams t ON m.away_team_api_id = t.api_id 
GROUP BY t.id

Обратите внимание! В данном запросе была использована группировка по столбцу id таблицы teams, хотя этот столбец не выводится в запросе. Это необходимо для того, чтобы команды с одинаковым названием, если такие найдутся, не группировались между собой. Группировка по названию команды в данном запросе будет неверной, так как есть несколько команд с одинаковым полным названием — мы говорили об этом в начале модуля.

Также, применяя агрегатные функции к соединённым таблицам, обращайте внимание на указание алиасов (или таблиц) при группировке и указании столбцов агрегатных функций. В нашей соединённой таблице есть два столбца с названием id, и если бы мы сформировали запрос без указания таблицы, как указано ниже, то...
система выдала бы уже знакомую нам ошибку: "... column "id" is ambiguous ...".

Мы можем использовать оператор HAVING для фильтрации сгруппированных данных.

Поставим задачу — вывести таблицу с суммарным количеством забитых голов в матчах по командам и сезонам для команд, в которых суммарное количество голов в матчах сезона больше 100.
Для начала посчитаем общее количество голов в матчах по сезонам.

SELECT
    m.season,
    SUM(m.home_team_goals) + SUM(m.away_team_goals) total_goals
FROM sql.matches m
GROUP BY m.season

Затем добавим таблицу с командами, группировку по командам и условие фильтрации.

SELECT
    m.season,
    t.long_name,
    SUM(m.home_team_goals) + SUM(m.away_team_goals) total_goals
FROM sql.matches m
JOIN sql.teams t ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id
GROUP BY m.season, t.id
HAVING SUM(m.home_team_goals) + SUM(m.away_team_goals) > 100

Напишите запрос, с помощью которого можно вывести список полных названий команд, сыгравших в гостях 150 и более матчей.
Отсортируйте список по названию команды.

SELECT
    t.long_name
FROM
    sql.matches m
JOIN sql.teams t ON m.away_team_api_id = t.api_id
GROUP BY t.id
HAVING COUNT(*) >= 150
ORDER BY 1

## 5. Способы соединения таблиц
### Операторы

### INNER JOIN

INNER JOIN — это тот же JOIN (слово inner в операторе можно опустить).
Для INNER JOIN работает следующее правило: присоединяются только те строки таблиц, которые удовлетворяют условию соединения. Если в любой из соединяемых таблиц находятся такие строки, которые не удовлетворяют заявленному условию, — они отбрасываются.

Рассмотрим на примере нашего датасета. 
SELECT 
COUNT(DISTINCT id)
FROM sql.teams

Теперь добавим к teams таблицу с матчами.

SELECT 
COUNT(DISTINCT t.id) 
FROM 
sql.teams t
JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id


И в таблице останется уже не 299 команд, а только 292.
?
Почему?

Дело в том, что таблица sql.matches по какой-то причине не содержит информацию о командах Lierse SK, KVC Westerlo, KAS Eupen, Club Brugge KV, KV Oostende, RSC Anderlecht и Hull City, зато они есть в таблице sql.teams. Возможно, эти команды не участвовали ни в одном матче или записи по этим матчам были удалены.

### LEFT OUTER JOIN и RIGHT OUTER JOIN

Также существуют схожие друг с другом типы соединения — LEFT JOIN и RIGHT JOIN (слово outer в операторе можно опустить).
Для LEFT JOIN работает следующее правило: из левой (относительно оператора) таблицы сохраняются все строки, а из правой добавляются только те, которые соответствуют условию соединения. Если в правой таблице не находится соответствия, то значения строк второй таблицы будут иметь значение NULL.

оставим следующую задачу: вывести полные названия команд, данных по которым нет в таблице matches.

Для начала посмотрим на результат запроса после соединения.

SELECT
    t.long_name,
    m.id
FROM sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id
ORDER BY m.id DESC

Теперь, чтобы выбрать такие команды, которые не принимали участия в матчах, достаточно добавить условие where m.id is null (или любое другое поле таблицы matches).

SELECT
    t.long_name
FROM 
    sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id
WHERE m.id IS NULL

Обратите внимание! Если мы добавим какой-либо фильтр по значению для таблицы matches, то LEFT JOIN превратится в INNER JOIN, поскольку для второй таблицы станет необходимым присутствие значения в строке.

__
Используя LEFT JOIN, выведите список уникальных названий команд, содержащихся в таблице matches. Отсортируйте список в алфавитном порядке.

SELECT
  DISTINCT t.long_name
FROM
  sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id or t.api_id = m.away_team_api_id
WHERE m.id is not null
ORDER BY 1
__

С LEFT JOIN также работают агрегатные функции, что позволяет не потерять значения из левой таблицы. Например, мы можем вывести сумму голов команд по гостевым матчам.

SELECT
    t.long_name,
    SUM(m.away_team_goals) total_goals
FROM   
    sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.away_team_api_id
GROUP BY t.id
ORDER BY 2 DESC

Обратите внимание! При применении функций SUM, MIN, MAX, AVG к полям со значением NULL в результате получится NULL, а не 0. А при использовании функции COUNT, наоборот, получится 0.
___
Используя LEFT JOIN, напишите запрос, который выведет полное название команды (long_name), количество матчей, в которых участвовала команда, — домашних и гостевых (matches_cnt).
Отсортируйте по количеству матчей в порядке возрастания, затем по названию команды в алфавитном порядке.
select
t.long_name,
count(m.id)
from
sql.teams t
left join sql.matches m on t.api_id = m.home_team_api_id or t.api_id = m.away_team_api_id
group by t.id
order by 2, 1
___

### FULL OUTER JOIN

Оператор FULL OUTER JOIN объединяет в себе LEFT и RIGHT JOIN и позволяет сохранить кортежи обеих таблиц. Даже если не будет соответствий, мы сохраним все записи из обеих таблиц.

FULL OUTER JOIN может быть полезен в ситуациях, когда схема данных недостаточно нормализована и не хватает таблиц-справочников.

Пример: в базе данных интернет-магазина есть две таблицы — с зарегистрированными пользователями и пользователями, оформившими заказ. При этом оформить заказ можно без регистрации, а зарегистрироваться — без оформления заказа.

Предположим, что вам необходимо получить полный список пользователей — и оформивших заказ, и зарегистрированных, — но в базе данных этой объединённой таблицы нет. В данном случае можно использовать FULL OUTER JOIN для получения полного списка, соединив таким образом таблицы c заказами и регистрациями по id пользователя.

### Cross Join

На самом деле с этим оператором соединения таблиц вы познакомились в самом начале текущего модуля.

CROSS JOIN соединяет таблицы так, что каждая запись в первой таблице присоединяется к каждой записи во второй таблице, иначе говоря, даёт декартово произведение.

мы можем получить все возможные комбинации полных названий команд в матчах.
SELECT
    DISTINCT
    t1.long_name home_team, 
    t2.long_name away_team
FROM
    sql.teams t1
    CROSS JOIN sql.teams t2

___

Напишите запрос, который выведет все возможные уникальные комбинации коротких названий домашней команды (home_team) и коротких названий гостевой команды (away_team).
Отсортируйте запрос по первому и второму столбцам.

SELECT
    DISTINCT
    t1.short_name home_team, 
    t2.short_name away_team
FROM
    sql.teams t1
    CROSS JOIN sql.teams t2
order by 1, 2

### NATURAL JOIN

Ключевое слово natural в начале оператора JOIN позволяет не указывать условие соединения таблиц — для соединения будут использованы столбцы с одинаковым названием из этих таблиц.

NATURAL JOIN можно использовать с любыми видами соединений, которые требуют условия соединения:

→ NATURAL INNER JOIN (возможна запись NATURAL JOIN);
→ NATURAL LEFT JOIN;
→ NATURAL RIGHT JOIN;
→ NATURAL FULL OUTER JOIN.

При использовании NATURAL JOIN прежде всего стоит обратить внимание на ключи таблиц. Для наших таблиц teams и matches этот вид соединения не подойдёт, так как общим для обеих таблиц является столбец id, но таблицы соединяются по другим столбцам.

При построении запроса с несколькими JOIN старайтесь идти слева направо. Сначала выберите таблицу, которая является центральной в соответствии с поставленной задачей, вопросом. Затем добавляйте таблицы поэтапно в зависимости от бизнес-логики запроса.

Например, для ответа на вопрос: «Какая команда сыграла больше всех матчей в сезоне 2010/2011?» в качестве центральной лучше выбрать таблицу с командами.

А для ответа на вопрос: «В каком сезоне участвовало больше всего команд?» — таблицу с матчами.

Стоит отметить, что из рассмотренных видов соединений чаще всего используются INNER JOIN и LEFT JOIN. Другие операторы используются реже, но стоит помнить об их существовании при решении нестандартных задач.